In [1]:
import pandas as pd
import os
import re
import nltk

In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

In [3]:
my_file = pd.read_csv("/home/bjit/testing_work/final__csv.csv")
print(my_file.head())

                                      URL                Family  Class
0  47faeb4f1b75a48499ba14e9b1cd895a.co.cc  bamital_dga_dbaa1822      1
1  47faeb4f1b75a48499ba14e9b1cd895a.cz.cc  bamital_dga_dbaa1822      1
2    9b86bb2ef4bad69cca0110076215e1f4.org  bamital_dga_dbaa1822      1
3   9b86bb2ef4bad69cca0110076215e1f4.info  bamital_dga_dbaa1822      1
4  9b86bb2ef4bad69cca0110076215e1f4.co.cc  bamital_dga_dbaa1822      1


In [4]:
my_file['URL'] = my_file['URL'].str.lower()

In [5]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n ):
    text = re.split(r"\W+",str(text))
    joined_text = " ".join(text)
#     return [ ' '.join(grams) for grams in n_grams]
    return joined_text

In [6]:
def apply_cleaning_function_to_list(X):
    cleaned_X = []
    for element in X:
        cleaned_X.append(get_ngrams(element,2))
    return cleaned_X

In [7]:
my_file['text'] = apply_cleaning_function_to_list(my_file['URL'])

In [8]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(my_file['text'], my_file['Class'])

In [9]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=5000)
tfidf_vect_ngram.fit(my_file['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [10]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [11]:
# accuracy = train_model(naive_bayes.GaussianNB(), xtrain_tfidf_ngram.toarray(), train_y, xvalid_tfidf_ngram.toarray())
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

NB, N-Gram Vectors:  0.8034132098268224
